In [1]:
from nb_space_restorer import NBSpaceRestorer
from tamil import utf8

In [2]:
mapping = {ch: chr(i + 40000) for i, ch in
           enumerate(sorted(utf8.tamil_letters.copy(), reverse=True, key=lambda x: len(x)))}
inverse_mapping = {v: k for k, v in mapping.items()}


def map(text):
    for k in mapping:
        text = text.replace(k, mapping[k])

    return text


def unmap(text):
    for k in inverse_mapping:
        text = text.replace(k, inverse_mapping[k])

    return text


def add_dollar(s):
    for i in range(len(s) - 1, 0, -1):
        if s[i] == ' ' or s[i - 1] == ' ':
            continue
        s = s[:i] + '$' + s[i:]

    return s

def get_annotation(pred, ref):
    ref_map = add_dollar(map(ref))
    pred_map = add_dollar(map(pred))
    final = pred_map

    for i in range(len(ref_map)):
        if ref_map[i] == ' ' or pred_map[i] == ' ':
            if ref_map[i] == ' ' and pred_map[i] == ' ':
                final = final[:i] + "#" + final[i + 1:]
            elif ref_map[i] == ' ' and pred_map[i] != ' ':
                # False negative
                final = final[:i] + "@" + final[i + 1:]
            elif ref_map[i] != ' ' and pred_map[i] == ' ':
                # False positive
                final = final[:i] + "!" + final[i + 1:]

    idx = 0

    lst = []

    for i in range(len(final)):

        elem = final[i]
        if elem == '#' or elem == '@' or elem == '!':
            lst.append(unmap(final[idx:i].replace('$', '')))
            idx = i + 1
            if elem == '#':
                lst.append("[TP]")
            elif elem == '@':
                lst.append("[FN]")
            elif elem == '!':
                lst.append("[FP]")

    lst.append(unmap(final[idx:].replace('$', '')))
    return ''.join(lst)

In [3]:
tamil_space_restorer = NBSpaceRestorer.load(
        '../pickle/AncientTamilSegmenter-v1.0.pickle',
        read_only=True
    )

Finished loading model.


In [5]:
reference = "நெல்லு நாற்கலமும் திருக்கண்ணாமடைக்கு வெண்டுவதவையிக்கு நெல்"
# reference = "நெல்லுப் பதக்கும் பெருனிருவமுதுக்கு அரிசி"
# reference = "இரண்டு ஒளிக்கதிர் களும் இரவில் வானத்தை நோக்கி பயணிக்கச் செய்யப்பட்டவை"
input_str = reference.replace(' ', '')

result = unmap(tamil_space_restorer.restore(map(input_str)))
evaluation = get_annotation(result, unmap(map(reference)))


print(f"Input: {input_str}")
print(f"Reference: {reference}")
print(f"Prediction: {evaluation}")

Input: நெல்லுநாற்கலமும்திருக்கண்ணாமடைக்குவெண்டுவதவையிக்குநெல்
Reference: நெல்லு நாற்கலமும் திருக்கண்ணாமடைக்கு வெண்டுவதவையிக்கு நெல்
Prediction: நெல்லு[FN]நாற்கலமும்[FN]திருக்கண்ணாமடைக்கு[TP]வெண்டுவதவையிக்கு[FN]நெல்
